# Market Risk - TD5

## Library importation

In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_excel("Dataset TD5.xlsx", header=None)

In [17]:
# Chargement du fichier SANS header


# --- Définition des colonnes de chaque série ---
gbp_cols = [0, 1, 2]
sek_cols = [4, 5, 6]
cad_cols = [8, 9, 10]

# --- Fonction de nettoyage d’un bloc ---
def clean_block(df, cols, prefix):
    """
    Nettoie un bloc (GBPEUR, SEKEUR, CADEUR) et renvoie un DataFrame propre.
    """
    block = df[cols].copy()

    # Ligne 1 contient les vrais noms de colonnes : Date / HIGH / LOW
    block.columns = block.iloc[1]

    # Suppression des deux premières lignes (nom + header)
    block = block.iloc[2:]

    # Renommage des colonnes
    block.columns = ["Date", f"{prefix}_HIGH", f"{prefix}_LOW"]

    # Nettoyage des types
    block["Date"] = pd.to_datetime(block["Date"], errors="coerce")
    block[f"{prefix}_HIGH"] = pd.to_numeric(block[f"{prefix}_HIGH"], errors="coerce")
    block[f"{prefix}_LOW"] = pd.to_numeric(block[f"{prefix}_LOW"], errors="coerce")

    # Suppression des lignes vides
    block = block.dropna(subset=["Date"])

    return block.reset_index(drop=True)

# --- Création des 3 DataFrames propres ---
df_gbp = clean_block(df, gbp_cols, "GBP")
df_sek = clean_block(df, sek_cols, "SEK")
df_cad = clean_block(df, cad_cols, "CAD")

C:\Users\natga\AppData\Local\Temp\ipykernel_14820\2396073099.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  block["Date"] = pd.to_datetime(block["Date"], errors="coerce")
C:\Users\natga\AppData\Local\Temp\ipykernel_14820\2396073099.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  block["Date"] = pd.to_datetime(block["Date"], errors="coerce")
C:\Users\natga\AppData\Local\Temp\ipykernel_14820\2396073099.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  block["Date"] = pd.to_datetime(block["Date"], errors="coerce")


In [18]:
def computeMean(df):
    df["mean"] = (df.iloc[:,1] + df.iloc[:,2])/2

def ComputeM2(df):
    return ((df["mean"]- df["mean"].shift(1)) * (df["mean"] - df["mean"].shift(1) )).mean()

def ComputeM2_prime(df):
    m2_prime = 0.0
        
    for i in range (1, (len(df)-1) // 2 + 1):
        m2_prime += (df["mean"].iloc[2*i] - df["mean"].iloc[2*i-2]) * (df["mean"].iloc[2*i] - df["mean"].iloc[2*i-2])
    m2_prime /= (len(df)-1) // 2
    
    return m2_prime

def ComputeH_hat(df):
    computeMean(df)
    m2_prime = ComputeM2_prime(df)
    m2 = ComputeM2(df)
    return np.log2(m2_prime/m2) / 2

print("df_gbp :", ComputeH_hat(df_gbp))
print("df_sek :", ComputeH_hat(df_sek))
print("df_cad :", ComputeH_hat(df_cad))

df_gbp : 0.6714143303551521
df_sek : 0.6545913434209916
df_cad : 0.6552439913405917


In [19]:
def ComputeReturn(df):
    df["return"] = df["mean"] / df["mean"].shift(1) - 1
    df.dropna(inplace = True)

def ComputeVolAnnualised(df):
    H = ComputeH_hat(df)
    ComputeReturn(df)
    return np.std(df["return"], ddof=1) * ((252*96)**H)

print(ComputeVolAnnualised(df_gbp))
print(ComputeVolAnnualised(df_sek))
print(ComputeVolAnnualised(df_cad))

0.546938318760874
0.24223410463728343
0.37736743674564216


## Wawelet

In [20]:
# Mother wavelet
def Wavelet_Haar(t):
    return np.where((0 <= t) & (t < 0.5), 1,
        np.where((0.5 <= t) & (t < 1), -1,
                 0)) 

# Daughter wavelet
def daughter_wavelet(j, k, t):
    return 2**(j/2) * Wavelet_Haar(t * 2**j - k)

# Wavelet transform
# z is the signal => price
def z_empirical(df, j, k, tau):
    t = np.arange(len(df)) * tau
    return np.sum(df["mean"] * daughter_wavelet(j, k, t) * tau)

couper les données pour en avoir une puissance de 2

ex : 8 => 2^3


In [21]:
df_gbp

,Date,GBP_HIGH,GBP_LOW,mean,return
1,2016-03-07 09:15:00,1.2940,1.2930,1.29350,0.000812
2,2016-03-07 09:30:00,1.2943,1.2922,1.29325,-0.000193
3,2016-03-07 09:45:00,1.2930,1.2913,1.29215,-0.000851
4,2016-03-07 10:00:00,1.2931,1.2921,1.29260,0.000348
5,2016-03-07 10:15:00,1.2926,1.2921,1.29235,-0.000193
...,...,...,...,...,...
12924,2016-09-07 17:00:00,1.1879,1.1867,1.18730,-0.000589
12925,2016-09-07 17:15:00,1.1883,1.1874,1.18785,0.000463
12926,2016-09-07 17:30:00,1.1880,1.1874,1.18770,-0.000126
12927,2016-09-07 17:45:00,1.1874,1.1866,1.18700,-0.000589


In [22]:
# x[start : stop : step]
def Scale_coefficient(df):
    C = df["mean"].to_numpy()
    step = len(C) // 2 * 2
    print(step)
    scale_total = [C]
    while len(C) > 1:
        C = (C[0:step:2] + C[1:step:2]) / np.sqrt(2)
        step/=2
        scale_total.append(C)
    
    return scale_total

print(Scale_coefficient(df_gbp))

12928


TypeError: slice indices must be integers or None or have an __index__ method

Attente du projet = consigne de garcin au dessus

Code + rapport
plus il a la vie facile meilleure est la note

rapport =  monterer

quelle donnée
modif sur les données
quelle formule
interprétation du résultat si demadné

mettre le code dans le rapport

moins il a a ouvrir le code mieux c'est

capture d'écran de code

bcp de points sur l'interprétation

utilisation des librairies : librairies de bases : pandas, numpy, maths

plus c'est détaillé mieux c'est

montrer qu'on a compris la démarche, maitriser le concepte

--

utilisation des LLM
pas une ligne de code ni mot par LLM => SINON ON EST MORT

